In [15]:
from time import time,sleep
from kafka import KafkaProducer
import json
import pandas as pd
import numpy as np

In [16]:
df = pd.read_csv(r'K:\BEST\Boot Camp\Zoom Camp\data-engineering-zoomcamp\06-streaming\pyflink\green_tripdata_2019-10.csv')

C:\Users\karee\AppData\Local\Temp\ipykernel_11340\2018684856.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'K:\BEST\Boot Camp\Zoom Camp\data-engineering-zoomcamp\06-streaming\pyflink\green_tripdata_2019-10.csv')


In [17]:
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2019-10-01 00:26:02,2019-10-01 00:39:58,N,1.0,112,196,1.0,5.88,18.0,0.50,0.5,0.00,0.0,NaN,0.3,19.30,2.0,1.0,0.0
1,1.0,2019-10-01 00:18:11,2019-10-01 00:22:38,N,1.0,43,263,1.0,0.80,5.0,3.25,0.5,0.00,0.0,NaN,0.3,9.05,2.0,1.0,0.0
2,1.0,2019-10-01 00:09:31,2019-10-01 00:24:47,N,1.0,255,228,2.0,7.50,21.5,0.50,0.5,0.00,0.0,NaN,0.3,22.80,2.0,1.0,0.0
3,1.0,2019-10-01 00:37:40,2019-10-01 00:41:49,N,1.0,181,181,1.0,0.90,5.5,0.50,0.5,0.00,0.0,NaN,0.3,6.80,2.0,1.0,0.0
4,2.0,2019-10-01 00:08:13,2019-10-01 00:17:56,N,1.0,97,188,1.0,2.52,10.0,0.50,0.5,2.26,0.0,NaN,0.3,13.56,1.0,1.0,0.0


In [18]:
df_new = df[['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'tip_amount']]
df_new.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,tip_amount
0,2019-10-01 00:26:02,2019-10-01 00:39:58,112,196,1.0,5.88,0.00
1,2019-10-01 00:18:11,2019-10-01 00:22:38,43,263,1.0,0.80,0.00
2,2019-10-01 00:09:31,2019-10-01 00:24:47,255,228,2.0,7.50,0.00
3,2019-10-01 00:37:40,2019-10-01 00:41:49,181,181,1.0,0.90,0.00
4,2019-10-01 00:08:13,2019-10-01 00:17:56,97,188,1.0,2.52,2.26


In [19]:
df_new = df_new.fillna({
    'passenger_count': 0,
    'trip_distance': 0.0,
    'tip_amount': 0.0
})

In [20]:
df_new[df_new['lpep_pickup_datetime']=='2019-10-01 00:28:00']


,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,tip_amount
387007,2019-10-01 00:28:00,2019-10-01 00:50:00,42,174,0.0,7.62,0.0


In [8]:
df_new['passenger_count'].dtype

dtype('float64')

In [17]:
def serialize(data):
    return json.dumps(data).encode('utf-8')

In [18]:
server = 'localhost:9092'

topic = 'green-trips'

producer = KafkaProducer(
    bootstrap_servers = [server],
    value_serializer = serialize
    )

In [19]:
data = df_new.to_dict(orient='records')

In [20]:
print(data[0])

{'lpep_pickup_datetime': '2019-10-01 00:26:02', 'lpep_dropoff_datetime': '2019-10-01 00:39:58', 'PULocationID': 112, 'DOLocationID': 196, 'passenger_count': 1.0, 'trip_distance': 5.88, 'tip_amount': 0.0}


In [21]:
t0 = time()

for message in data:
    producer.send(topic, value=message)
    
producer.flush()

t1 = time()
took = t1 - t0
print("The total time taken is: ", took)

The total time taken is:  90.93317198753357


---

In [21]:
lookup_df= pd.read_csv(r'K:\BEST\Boot Camp\Zoom Camp\data-engineering-zoomcamp\06-streaming\pyflink\taxi_zone_lookup.csv')

In [22]:
lookup_df.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [42]:
lookup_df[lookup_df['LocationID']==95]

,LocationID,Borough,Zone,service_zone
94,95,Queens,Forest Hills,Boro Zone
